In [66]:
.ORIG x3000

    BRnzp ASCII_CONVERT
    
;; ------------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;; ------------------------------------------------------------------

;; Subroutine to convert binary value into ASCII decimal and print.

ASCII_CONVERT:
    LD R0, CONVERT_VAL
    ST R0, TEMPX
    ST R0, TEMPY
    
ASCII_BASE: ;; Base case: if val < #10, add x30 and print
    ADD R1, R0, #-10
    BRzp ASCII_LOOP ;; if val >= #10, skip to main loop
    
    LD R1, ASCII_POS
    ADD R0, R0, R1
    OUT R0
    HALT
    
ASCII_LOOP: ;; Non-base case
    
    LD R0, TEMPZ ;; R0 <- z
    LD R1, TEN ;; R1 <- 10
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1 = z*10
    ST R2, TEMPZ
    
    LD R0, TEMPY ;; R0 <- y
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1 = y/10
    
    BRp ASCII_LOOP ;; if quotient is positive, repeat loop
    
    ;; else if quotient is zero, use PREVIOUS quotient (current val of TEMPY) as y
    
    
    BRnzp ASCII_BASE ;; return to ASCII_BASE to check for base case
    
    
ASCII_POS:   .FILL x30
CONVERT_VAL:   .FILL #2 ;; value to be converted
ONE:   .FILL #1
TEN:   .FILL #10
TEMPX:   .FILL #0
TEMPY:   .FILL #0
TEMPZ:   .FILL #1

;;-----------------------------------------------------------------
;; Signposts to PUSH and POP

PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [71]:
%exe

Computation completed
Instructions: 2
Cycles: 19 (0.000010 milliseconds)

Registers:
PC: x048E
N: 1 Z: 0 P: 0 
R0: xFFFD R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3002 


In [51]:
%pc x3000


Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0032 R1: x003A R2: x0000 R3: x0000 
R4: x0000 R5: x3002 R6: x30B4 R7: x3094 


In [65]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x308F
(14/114) ADD R1, R1, #10 [60] (x308F*: x126A)
    R1 <= x003A
    NZP <= (0, 0, 1)

Registers:
PC: x308F
N: 0 Z: 0 P: 1 
R0: x0032 R1: x003A R2: x0000 R3: x0000 
R4: x0000 R5: x3002 R6: x30B4 R7: x308E 


In [74]:
.ORIG x3000

ADD R0, R0, #-3
ADD R1, R1, #2
ST R0, DATA

HALT

DATA: .FILL #-3

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [75]:
%exe

Computation completed
Instructions: 4
Cycles: 30 (0.000015 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: xFFFD R1: x0002 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x3004 
